# 📘 Day 3 – Co-visitation Matrix & Candidate Expansion

##  Introduction

In Day 3, we will build co-visitation matrices, a common technique in recommendation systems.
The idea:

If two items frequently appear together in the same user session,
they may be similar or often co-purchased/ co-clicked.

This day prepares candidates for retrieval (recall) in the next steps.

# 📦 1. Load Data (Chunk Loading for Safety)

In [1]:
import pandas as pd
import pyarrow.parquet as pq

path= "/kaggle/input/otto-recsys-dataset/train.parquet"
pf=pq.ParquetFile(path)

dfs=[]
for i,batch in enumerate(pf.iter_batches(batch_size=200000)):
    df = batch.to_pandas()
    dfs.append(df)
    print(f"Loaded chunk {i+1}")
    if i ==9:
        break

events =pd.concat(dfs, ignore_index=True)
events.head()

Loaded chunk 1
Loaded chunk 2
Loaded chunk 3
Loaded chunk 4
Loaded chunk 5
Loaded chunk 6
Loaded chunk 7
Loaded chunk 8
Loaded chunk 9
Loaded chunk 10


,aid,ts,type,session
0,1517085,1659304800025,0,0
1,1563459,1659304904511,0,0
2,1309446,1659367439426,0,0
3,16246,1659367719997,0,0
4,1781822,1659367871344,0,0


## 🔧 2. Preprocess Data

In [2]:
events =events.sort_values(["session","ts"])
events["ts"]=events["ts"].astype("int64")
events.head()

,aid,ts,type,session
0,1517085,1659304800025,0,0
1,1563459,1659304904511,0,0
2,1309446,1659367439426,0,0
3,16246,1659367719997,0,0
4,1781822,1659367871344,0,0


## 🧩 3. Build Co-visitation Matrix (Core Part)

•	 In a session, if commodity A and commodity B appear in the time window → consider that A and B have a co-present relationship.

•	 Count all sessions such pairs

•	 Finally, get a set of top-related products B for each A.

## 3.1 Define Helper Function

In [19]:
from collections import defaultdict
import numpy as np

def build_covis_matrix(df,event_type=None, time_window=30*60):
    """
    df:session-level dataframe
    event_type:'clicks','carts','orders',or None for all
    time_window:window in seconds for co-visitation
    """

    if event_type:
        df=df[df["type"] == event_type]

    session_groups =df.groupby("session")

    covis =defaultdict(lambda:defaultdict(float))

    for session_id, group in session_groups:
        item = group[["aid","ts"]].values

        for i in range(len(items)):
            aid_i,ts_i =items[i]

            for j in range(i+1,len(items)):
                aid_j,ts_j = items[j]

                if abs(ts_i - ts_j) <= time_window:

                    weight = 1.0
                    if event_type == "orders":
                        weight = 3.0
                    elif event_type == "carts":
                        weight = 2.0

                    covis[aid_i][aid_j] += weight
                    covis[aid_j][aid_i] += weight

    return covis

## 3.2 Build Co-visitation Matrics for All Types

In [20]:
covis_clicks = build_covis_matrix(events,event_type="clicks")
covis_carts = build_covis_matrix(events,event_type="carts")
covis_orders=build_covis_matrix(events,event_type="orders")

print("Done building co-visitation matrices.")

Done building co-visitation matrices.


## 🏆 4. Candidate Expansion (Top-40 Candidates)

In [21]:
def extract_topk(covis_dict,top_k=40):
    topk ={}
    for aid,neighbors in covis_dict.items():
        sorted_items = sorted(neighbors.items(),key=lambda x: -x[1])
        topk[aid] = [nid for nid, score in sorted_items[:top_40]]
    return topk

candidates_clicks = extract_topk(covis_clicks)
candidates_carts = extract_topk(covis_carts)
candidates_orders = extract_topk(covis_orders)

print("Candidate extraction completed.")

Candidate extraction completed.


## 💾 5. Merge Candidates (Final Expansion Dictionary)

In [22]:
final_candidates= {}

for dict_src in [candidates_clicks,candidates_carts,candidates_orders]:
    for aid,items in dict_src.items():
        if aid not in final_candidates:
            final_candidates[aid] = set()
        final_candidates[aid].update(items)

#Converted list
final_candidates ={k:list(v) for k ,v in final_candidates.items()}

print("Total items with candidates:",len(final_candidates))

Total items with candidates: 0


In [23]:
import json

with open("day3_candidates.parquet","w")as f:
    json.dump(final_candidates,f)

print("Saved day3_candidates.parquet")

Saved day3_candidates.parquet


## 🟦 Day 4 — Feature Engineering

## The main goals of this Day:

✔ Generate item-level / session-level characteristics required for training

✔ Calculate click/add purchase/purchase related features for each candidate item

✔ Convert time characteristics (hour, weekday, time decay)

✔ Merge into the final training sample (session × candidate item)

## Set Up

In [8]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

## 📚 1.Load Candidate Data

In [9]:
#Read the Candidate items dictionary generated by day3
with open("day3_candidates.parquet","r")as f:
    candidates = json.load(f)

print("Total sessions:",len(candidates))

Total sessions: 0


## 📕 2. Load Training Events

In [10]:
# Read training data（such as parquet）
events = pd. read_parquet("/kaggle/input/otto-recsys-dataset/train.parquet")

events.head()

,aid,ts,type,session
0,1517085,1659304800025,0,0
1,1563459,1659304904511,0,0
2,1309446,1659367439426,0,0
3,16246,1659367719997,0,0
4,1781822,1659367871344,0,0


## 📖 3. Basic Preprocessing


In [11]:
# Map the number to type(click=0,carts=1,orders=2)
type_map={"clicks":0,"carts":1,"orders":2}
events["type_code"]=events["type"].map(type_map)

#Convert the timestamp to hour and weekday
events["ts"]=events["ts"]//1000
events["dt"]=pd.to_datetime(events["ts"],unit="s")

events["hours"]= events["dt"].dt.hour
events["weekday"]=events["dt"].dt.weekday

## 4.Item-Level Features

## We calculate for each item (aid):

✔ Number of clicks

✔ The number of additional purchases

✔ Number of purchases

✔ Total number of interactions

✔ Click / Add Purchase / Purchase Ratio

✔ The position that appears in the average session (the more back, the higher the intention)

## 💾 4.1 Count-based item features

In [12]:
#Count the behavior count of each aid
item_stats = events.pivot_table(
    index="aid",
    columns="type",
    values="session",
    aggfunc="count",
    fill_value=0
)

item_stats.columns = ["clicks","carts","orders"]

item_stats["total_events"]=(
    item_stats["clicks"]+
    item_stats["carts"]+
    item_stats["orders"]
)

item_stats.head()

,clicks,carts,orders,total_events
aid,,,,
0,44,0,0,44
1,33,1,0,34
2,17,0,0,17
3,2333,186,38,2557
4,204,9,0,213


## 💾 4.2 Interaction ratios

In [13]:
item_stats["cart_ratio"]=item_stats["carts"]/(item_stats["total_events"]+1)
item_stats["order_ration"]=item_stats["orders"]/(item_stats["total_events"]+1)

## 📔 4.3 Item popularity(overall)

In [14]:
#Item popularity = number of item occurrences/number of all events
total_events = len(events)
item_stats["popularity"]= item_stats["total_events"]/total_events

## 📘 5.Session-Level Features

## The characteristics include:

✔ Time span in session

✔ The average interval of session

✔ The number of events in session

✔ The time of the last action

✔ Main behavior types



In [15]:
 import pandas as pd

df = pd.read_parquet("/kaggle/input/otto-recsys-dataset/train.parquet",
                     columns=["aid", "type", "ts"])

item_features = (
    df
    .groupby("aid")
    .agg(
        click_cnt = ("type", lambda x: (x=="clicks").sum()),
        cart_cnt  = ("type", lambda x: (x=="carts").sum()),
        order_cnt = ("type", lambda x: (x=="orders").sum()),
        last_ts   = ("ts", "max")
    )
    .reset_index()
)

item_features["cart_rate"] = item_features["cart_cnt"] / (item_features["click_cnt"] + 1)
item_features["order_rate"] = item_features["order_cnt"] / (item_features["click_cnt"] + 1)

item_features.head()

,aid,click_cnt,cart_cnt,order_cnt,last_ts,cart_rate,order_rate
0,0,0,0,0,1661712193846,0.0,0.0
1,1,0,0,0,1661422366083,0.0,0.0
2,2,0,0,0,1661604076124,0.0,0.0
3,3,0,0,0,1661722645019,0.0,0.0
4,4,0,0,0,1661715232131,0.0,0.0


## 📕 6.  Time Decay Feature

In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
events = pd. read_parquet("/kaggle/input/otto-recsys-dataset/train.parquet")

In [17]:
#Calculate time decay for each event
session_end_ts = events.groupby("session")["ts"].transform("max")
events["time_decay"]=np.exp(-0.01 * (session_end_ts - events["ts"]))

## 📚 7. Build Training Samples(session * candidate item)

In [8]:
df = pd.read_parquet(
    "/kaggle/input/otto-recsys-dataset/train.parquet",
    columns=["session", "aid", "ts"]
)

df = df.sort_values(["session", "ts"], kind="mergesort")

last_item = df.drop_duplicates("session", keep="last")[["session", "aid"]]
last_item.head()

,session,aid
275,0,161938
307,1,497868
340,2,672473
566,3,925352
585,4,479396


##  8.Merge Features

In [12]:
# 8. Merge Features and Generate Labels

import pandas as pd
import gc

# Load candidates from previous step
candidate_data = pd.read_parquet("/kaggle/input/otto-recsys-dataset/test.parquet")
print(f"Loaded {len(candidate_data):,} candidates")

# Load training events from Day 1
train_events = pd.read_parquet("/kaggle/input/otto-recsys-dataset/train.parquet")

# Sort to ensure correct order
train_events = train_events.sort_values(["session", "ts"]).reset_index(drop=True)

# Extract the last aid for each session (ground truth for "next item")
session_last_aid = train_events.groupby("session")["aid"].last()

# Convert to dictionary for fast lookup
last_aid_dict = session_last_aid.to_dict()

# Map the last aid to each candidate and create label
candidate_data["label"] = candidate_data.apply(
    lambda row: 1 if row["aid"] == last_aid_dict.get(row["session"], None) else 0,
    axis=1
)

print(f"Training dataset created: {len(candidate_data):,} samples")
print(f"Positive labels: {candidate_data['label'].sum():,}")
print(f"Positive rate: {candidate_data['label'].mean():.4f}")

# Save labeled training data
candidate_data.to_parquet("/kaggle/input/otto-recsys-dataset/test.parquet", index=False)
print("Labeled training data saved as test.parquet")

gc.collect()

Loaded 6,928,123 candidates
Training dataset created: 6,928,123 samples
Positive labels: 0
Positive rate: 0.0000
Labeled training data saved as training_data_labeled.parquet


33

##  9.  Generate Labels

In [21]:
# 9. Generate Labels

import pandas as pd
import gc

# Load candidates from previous step
candidate_data = pd.read_parquet("/kaggle/input/otto-recsys-dataset/test.parquet")  # Adjust if filename differs
print(f"Loaded {len(candidate_data):,} candidates")

# Load exploded training events
train_events = pd.read_parquet("/kaggle/input/otto-recsys-dataset/train.parquet")

# Sort to ensure chronological order
train_events = train_events.sort_values(["session", "ts"]).reset_index(drop=True)

# Extract the last aid for each session (ground truth)
last_aid_per_session = train_events.groupby("session")["aid"].last()
last_aid_dict = last_aid_per_session.to_dict()

# Create label using dictionary lookup
candidate_data["label"] = candidate_data.apply(
    lambda row: 1 if row["aid"] == last_aid_dict.get(row["session"]) else 0,
    axis=1
)

print(f"Labeled dataset created: {len(candidate_data):,} samples")
print(f"Positive labels: {candidate_data['label'].sum():,}")
print(f"Positive rate: {candidate_data['label'].mean():.4f}")

# Save labeled data for training
candidate_data.to_parquet("/kaggle/working/labeled_training_data.parquet", index=False)
print("Labeled training data saved as labeled_training_data.parquet")

gc.collect()

Loaded 6,928,123 candidates
Labeled dataset created: 6,928,123 samples
Positive labels: 0
Positive rate: 0.0000
Labeled training data saved as labeled_training_data.parquet


39

## 10.Save Feature Dataset 

In [22]:
train_df.to_parquet("day4_features.parquet")

print("Saved day4_features.parquet")

Saved day4_features.parquet
